## 1. 데이터 로드

In [30]:
import pandas as pd
import numpy as np
from pathlib import Path
import re

# 데이터 로드
data_dir = Path('Backend/target/classes/data')
law_df = pd.read_csv(data_dir / '법규조례_final.csv')
land_df = pd.read_csv(data_dir / '토지특성정보_혼합_final_2.csv')

print("✅ 데이터 로드 완료")
print(f"Law 원본: {len(law_df):,}건")
print(f"LandChar: {len(land_df):,}건")
print(f"\nLaw 컬럼: {list(law_df.columns)}")

✅ 데이터 로드 완료
Law 원본: 913,591건
LandChar: 5,841,461건

Law 컬럼: ['구분코드', '법률명', '토지이용명', '가능여부', '조건제한예외사항', '용도지역지구명']


## 2. 전처리 작업

### 2-1. Law 테이블에 지역명 추가

In [31]:
print("=" * 80)
print("🔧 전처리 1: 지역명 매핑 추가")
print("=" * 80)

# 법정동코드 파일 로드
address_file = Path('python/address.txt')
address_df = pd.read_csv(address_file, sep='\t', dtype={'법정동코드': str}, encoding='utf-8')

print(f"\n법정동코드 파일 로드: {len(address_df):,}건")

# 구분코드 체계 설명
print("\n구분코드 체계:")
print("  - 0: 전국 (국가 법령)")
print("  - 앞 2자리: 시도 (11 = 서울)")
print("  - 앞 5자리: 시군구 (11110 = 서울 종로구)")
print("  - 10자리: 법정동 (1111010100 = 서울 종로구 청운동)")

# Law 테이블의 구분코드 확인
print(f"\nLaw 테이블 구분코드 샘플:")
print(law_df['구분코드'].head(10))

# 5자리 코드 → 지역명 매핑 생성
region_mapping = {}

# 특수 케이스: 0 = 전국
region_mapping[0] = "전국"

# 폐지되지 않은 데이터만 사용
address_active = address_df[address_df['폐지여부'] == '존재'].copy()

# 모든 법정동코드에서 앞 5자리 추출
address_active['구분코드_5자리'] = address_active['법정동코드'].str[:5].astype(int)

# 각 5자리 코드별로 가장 짧은 지역명 찾기 (시도 또는 시군구 레벨)
for code in address_active['구분코드_5자리'].unique():
    if code == 0:  # 0은 이미 처리됨
        continue
    
    code_rows = address_active[address_active['구분코드_5자리'] == code]
    
    # 법정동명 길이 순으로 정렬 (짧은 것이 상위 행정구역)
    shortest_name = code_rows['법정동명'].str.len().idxmin()
    region_mapping[code] = code_rows.loc[shortest_name, '법정동명']

print(f"\n지역명 매핑 생성: {len(region_mapping):,}개")
print(f"\n샘플 매핑:")
for i, (code, region) in enumerate(list(region_mapping.items())[:15]):
    print(f"  {code} → {region}")

# Law 테이블에 지역명 컬럼 추가
law_df['지역명'] = law_df['구분코드'].map(region_mapping)

# 매핑 성공률 확인
mapped_count = law_df['지역명'].notna().sum()
print(f"\n✅ 지역명 매핑 완료: {mapped_count:,}/{len(law_df):,}건 ({mapped_count/len(law_df)*100:.1f}%)")

# 매핑 실패 케이스 확인
unmapped = law_df[law_df['지역명'].isna()]
if len(unmapped) > 0:
    print(f"\n⚠️ 매핑 실패: {len(unmapped):,}건")
    print(f"실패한 구분코드 샘플: {unmapped['구분코드'].unique()[:10]}")
    
    # 실패한 코드가 address.txt에 있는지 확인
    failed_codes = unmapped['구분코드'].unique()
    print(f"\n실패한 코드 분석:")
    for fc in failed_codes[:5]:
        if fc == 0:
            print(f"  {fc}: 전국 (이미 처리됨)")
        else:
            matching = address_active[address_active['구분코드_5자리'] == fc]
            if len(matching) > 0:
                print(f"  {fc}: address.txt에 {len(matching)}건 존재 - {matching['법정동명'].iloc[0]}")
            else:
                print(f"  {fc}: address.txt에 없음")
else:
    print("\n✅ 모든 구분코드가 정상 매핑되었습니다!")


🔧 전처리 1: 지역명 매핑 추가

법정동코드 파일 로드: 50,085건

구분코드 체계:
  - 0: 전국 (국가 법령)
  - 앞 2자리: 시도 (11 = 서울)
  - 앞 5자리: 시군구 (11110 = 서울 종로구)
  - 10자리: 법정동 (1111010100 = 서울 종로구 청운동)

Law 테이블 구분코드 샘플:
0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: 구분코드, dtype: int64

지역명 매핑 생성: 285개

샘플 매핑:
  0 → 전국
  11000 → 서울특별시
  11110 → 서울특별시 종로구
  11140 → 서울특별시 중구
  11170 → 서울특별시 용산구
  11200 → 서울특별시 성동구
  11215 → 서울특별시 광진구
  11230 → 서울특별시 동대문구
  11260 → 서울특별시 중랑구
  11290 → 서울특별시 성북구
  11305 → 서울특별시 강북구
  11320 → 서울특별시 도봉구
  11350 → 서울특별시 노원구
  11380 → 서울특별시 은평구
  11410 → 서울특별시 서대문구

✅ 지역명 매핑 완료: 913,591/913,591건 (100.0%)

✅ 모든 구분코드가 정상 매핑되었습니다!


### 2-2. 용도지역명 정규화

In [32]:
print("=" * 80)
print("🔧 전처리 2: 용도지역명 정규화 (스킵)")
print("=" * 80)

# 실제 데이터에 정규화가 필요한 케이스가 없음을 확인
# 원본 용도지역지구명을 그대로 사용
print("\n⚠️ 용도지역 정규화 스킵")
print("  - 이유: 실제 데이터에 정규화가 필요한 값이 없음")
print("  - 원본 '용도지역지구명' 컬럼을 그대로 사용")

# 상위 용도지역 분포
print(f"\n용도지역 분포 (상위 10개):")
print(law_df['용도지역지구명'].value_counts().head(10))


🔧 전처리 2: 용도지역명 정규화 (스킵)

⚠️ 용도지역 정규화 스킵
  - 이유: 실제 데이터에 정규화가 필요한 값이 없음
  - 원본 '용도지역지구명' 컬럼을 그대로 사용

용도지역 분포 (상위 10개):
용도지역지구명
준주거지역        23521
제1종일반주거지역    23364
자연녹지지역       23275
일반상업지역       23235
중심상업지역       22955
보전녹지지역       22932
준공업지역        22709
제3종일반주거지역    22658
일반공업지역       22658
근린상업지역       22607
Name: count, dtype: int64


### 2-3. 토지이용명 키워드 추출

In [33]:
print("=" * 80)
print("🔧 전처리 3: 토지이용명 키워드 추출 (스킵)")
print("=" * 80)

# 괄호 제거 후 감소율이 0.8%로 미미함
# 원본 토지이용명을 그대로 사용
print("\n⚠️ 토지이용 키워드 추출 스킵")
print("  - 이유: 괄호 제거 후 감소율이 0.8%로 미미함")
print("  - 원본 '토지이용명' 컬럼을 그대로 사용")

# 상위 토지이용 분포
print(f"\n토지이용 분포 (상위 20개):")
print(law_df['토지이용명'].value_counts().head(20))


🔧 전처리 3: 토지이용명 키워드 추출 (스킵)

⚠️ 토지이용 키워드 추출 스킵
  - 이유: 괄호 제거 후 감소율이 0.8%로 미미함
  - 원본 '토지이용명' 컬럼을 그대로 사용

토지이용 분포 (상위 20개):
토지이용명
제과점, 휴게음식점                                   12966
단란주점                                         12482
극장, 비디오물감상실, 비디오물소극장, 서커스장, 연예장, 영화관, 음악당    11717
교회, 기도원, 사당, 사찰, 성당, 수녀원, 수도원, 제실            11698
다중생활시설                                       11641
결혼상담소, 금융업소, 부동산중개사무소, 사무소, 출판사              11493
야영장시설                                        11357
공장                                            7094
봉안당                                           6885
창고                                            6665
하역장                                           6641
세차장                                           6603
직업훈련소                                         6566
집배송시설                                         6548
축사                                            6542
학원                                            6541
도료류판매소                                        6521
주차장  

### 2-4. 가능여부 정규화

In [34]:
print("=" * 80)
print("🔧 전처리 4: 가능여부 정규화")
print("=" * 80)

def normalize_permission(text):
    """
    가능여부를 3가지 카테고리로 정규화
    - 가능: 허가, 가능, 허용, 건축 등
    - 불가: 불가, 금지, 제한 등
    - 조건부: 조건, 단서 등
    - 알수없음: 기타
    """
    if pd.isna(text):
        return "알수없음"
    
    text = str(text).lower()
    
    # 긍정 키워드
    if any(kw in text for kw in ['허가', '가능', '허용', '건축가능']):
        return "가능"
    # 부정 키워드
    elif any(kw in text for kw in ['불가', '금지', '제한', '불허']):
        return "불가"
    # 조건부 키워드
    elif any(kw in text for kw in ['조건', '단서', '예외']):
        return "조건부"
    else:
        return "알수없음"

# 가능여부 정규화 적용
law_df['가능여부_정규화'] = law_df['가능여부'].apply(normalize_permission)

# 원본 가능여부 분포
print(f"\n원본 가능여부 종류: {law_df['가능여부'].nunique()}개")
print("\n원본 상위 15개:")
print(law_df['가능여부'].value_counts().head(15))

# 정규화 후 분포
print(f"\n\n✅ 가능여부 정규화 완료")
print("\n정규화 결과:")
print(law_df['가능여부_정규화'].value_counts())
print(f"\n비율:")
print(law_df['가능여부_정규화'].value_counts(normalize=True).mul(100).round(1))

🔧 전처리 4: 가능여부 정규화

원본 가능여부 종류: 477개

원본 상위 15개:
가능여부
건축가능        561735
건축금지        337521
설치가능          3856
행위가능           979
형질변경가능         634
행위금지           444
증축가능           431
형질변경금지         375
신축·증축금지        320
개축.재축금지        313
설치금지           309
용도변경금지         301
이전금지           296
사육금지           258
신설, 증설금지       257
Name: count, dtype: int64


✅ 가능여부 정규화 완료

정규화 결과:
가능여부_정규화
가능      570926
불가      342605
알수없음        60
Name: count, dtype: int64

비율:
가능여부_정규화
가능      62.5
불가      37.5
알수없음     0.0
Name: proportion, dtype: float64


## 3. 전처리 결과 검증

In [35]:
print("=" * 80)
print("✅ 전처리 결과 검증")
print("=" * 80)

# 새로 추가된 컬럼 확인
new_columns = ['지역명', '가능여부_정규화']

print("\n📊 새로운 컬럼 NULL 체크:")
for col in new_columns:
    null_count = law_df[col].isna().sum()
    null_pct = null_count / len(law_df) * 100
    print(f"  {col:20s}: NULL {null_count:>8,}건 ({null_pct:>5.1f}%)")

# 전체 컬럼 리스트
print(f"\n📋 전처리 후 전체 컬럼 ({len(law_df.columns)}개):")
for i, col in enumerate(law_df.columns, 1):
    print(f"  {i:2d}. {col}")

# 데이터 샘플 확인
print(f"\n📝 샘플 데이터 (5건):")
display_cols = ['지역명', '용도지역지구명', '토지이용명', '가능여부_정규화', '조건제한예외사항']
print(law_df[display_cols].head(5))


✅ 전처리 결과 검증

📊 새로운 컬럼 NULL 체크:
  지역명                 : NULL        0건 (  0.0%)
  가능여부_정규화            : NULL        0건 (  0.0%)

📋 전처리 후 전체 컬럼 (8개):
   1. 구분코드
   2. 법률명
   3. 토지이용명
   4. 가능여부
   5. 조건제한예외사항
   6. 용도지역지구명
   7. 지역명
   8. 가능여부_정규화

📝 샘플 데이터 (5건):
  지역명 용도지역지구명         토지이용명 가능여부_정규화 조건제한예외사항
0  전국  산업정비구역  건축물(가설건축물포함)       가능      NaN
1  전국  산업정비구역           공작물       가능      NaN
2  전국  산업정비구역            토지       가능      NaN
3  전국  산업정비구역            토지       가능      NaN
4  전국  산업정비구역          공유수면       가능      NaN


## 4. 전처리 데이터 저장

In [36]:
print("=" * 80)
print("💾 전처리 데이터 저장")
print("=" * 80)

# 최종 컬럼 선택 (7개)
final_cols = [
    '구분코드', '지역명', '법률명',
    '용도지역지구명', '토지이용명',
    '가능여부_정규화', '조건제한예외사항'
]

# 최종 데이터프레임 생성
law_final = law_df[final_cols].copy()

# 저장할 파일명
output_file = data_dir / '법규조례_전처리완료.csv'

# CSV 저장
law_final.to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"\n✅ 저장 완료: {output_file}")
print(f"\n파일 정보:")
print(f"  - 레코드 수: {len(law_final):,}건")
print(f"  - 컬럼 수: {len(law_final.columns)}개")
print(f"  - 파일 크기: {output_file.stat().st_size / 1024 / 1024:.1f} MB")

print(f"\n📋 최종 컬럼 목록:")
for i, col in enumerate(final_cols, 1):
    print(f"  {i}. {col}")


💾 전처리 데이터 저장

✅ 저장 완료: Backend\target\classes\data\법규조례_전처리완료.csv

파일 정보:
  - 레코드 수: 913,591건
  - 컬럼 수: 7개
  - 파일 크기: 192.1 MB

📋 최종 컬럼 목록:
  1. 구분코드
  2. 지역명
  3. 법률명
  4. 용도지역지구명
  5. 토지이용명
  6. 가능여부_정규화
  7. 조건제한예외사항


## 5. 전처리 효과 확인

In [37]:
print("=" * 80)
print("📈 전처리 효과 확인")
print("=" * 80)

# 시나리오 1: 지역명으로 직접 검색
print("\n🔍 시나리오 1: '강남구' 검색")
test_region = "강남구"
result1 = law_df[law_df['지역명'].str.contains(test_region, na=False)]
print(f"  - 검색 결과: {len(result1):,}건")
if len(result1) > 0:
    print(f"  - 용도지역 (상위 5개): {result1['용도지역지구명'].value_counts().head(5).to_dict()}")

# 시나리오 2: 용도지역 + 토지이용
print("\n🔍 시나리오 2: '제1종일반주거지역 + 음식점'")
result2 = law_df[
    (law_df['용도지역지구명'] == '제1종일반주거지역') &
    (law_df['토지이용명'].str.contains('음식점', na=False))
]
print(f"  - 검색 결과: {len(result2):,}건")
if len(result2) > 0:
    print(f"  - 가능여부 분포:")
    print(result2['가능여부_정규화'].value_counts())

# 시나리오 3: 토지이용 키워드 검색
print("\n🔍 시나리오 3: '단독주택' 키워드 검색")
result3 = law_df[
    (law_df['토지이용명'].str.contains('단독주택', na=False)) &
    (law_df['가능여부_정규화'] == '가능')
]
print(f"  - 검색 결과: {len(result3):,}건")
if len(result3) > 0:
    print(f"  - 가능한 용도지역 (상위 10개):")
    print(result3['용도지역지구명'].value_counts().head(10))

print("\n" + "=" * 80)
print("✅ 전처리 완료! 이제 챗봇에서 사용 가능합니다.")
print("=" * 80)


📈 전처리 효과 확인

🔍 시나리오 1: '강남구' 검색
  - 검색 결과: 0건

🔍 시나리오 2: '제1종일반주거지역 + 음식점'
  - 검색 결과: 482건
  - 가능여부 분포:
가능여부_정규화
가능    481
불가      1
Name: count, dtype: int64

🔍 시나리오 3: '단독주택' 키워드 검색
  - 검색 결과: 5,475건
  - 가능한 용도지역 (상위 10개):
용도지역지구명
준주거지역        163
제1종전용주거지역    162
제2종전용주거지역    162
보전녹지지역       162
제2종일반주거지역    161
자연녹지지역       161
제1종일반주거지역    156
근린상업지역       156
일반상업지역       156
제3종일반주거지역    153
Name: count, dtype: int64

✅ 전처리 완료! 이제 챗봇에서 사용 가능합니다.


## 6. 토지특성정보 분석

In [38]:
print("=" * 80)
print("📊 토지특성정보 데이터 분석")
print("=" * 80)

print(f"\n기본 정보:")
print(f"  - 레코드 수: {len(land_df):,}건")
print(f"  - 컬럼 수: {len(land_df.columns)}개")

print(f"\n📋 컬럼 목록:")
for i, col in enumerate(land_df.columns, 1):
    print(f"  {i:2d}. {col}")

print(f"\n📊 각 컬럼의 고유값 개수 및 NULL 비율:")
for col in land_df.columns:
    unique_count = land_df[col].nunique()
    null_count = land_df[col].isna().sum()
    null_pct = null_count / len(land_df) * 100
    print(f"  {col:30s}: 고유값 {unique_count:>6,}개 | NULL {null_count:>6,}건 ({null_pct:>5.1f}%)")

print(f"\n📝 샘플 데이터 (처음 3건):")
print(land_df.head(3).T)  # 전치하여 컬럼별로 보기

print(f"\n🔍 각 컬럼의 데이터 타입 및 샘플값:")
for col in land_df.columns:
    sample_values = land_df[col].dropna().head(5).tolist()
    print(f"\n{col}:")
    print(f"  - 타입: {land_df[col].dtype}")
    print(f"  - 샘플: {sample_values}")


📊 토지특성정보 데이터 분석

기본 정보:
  - 레코드 수: 5,841,461건
  - 컬럼 수: 14개

📋 컬럼 목록:
   1. 법정동코드
   2. 법정동명
   3. 대장구분명
   4. 지번
   5. 지목명
   6. 토지면적
   7. 용도지역명1
   8. 용도지역명2
   9. 토지이용상황
  10. 지형높이
  11. 지형형상
  12. 도로접면
  13. query_key
  14. 구분코드

📊 각 컬럼의 고유값 개수 및 NULL 비율:
  법정동코드                         : 고유값  2,471개 | NULL      0건 (  0.0%)
  법정동명                          : 고유값  2,471개 | NULL      0건 (  0.0%)
  대장구분명                         : 고유값      9개 | NULL      0건 (  0.0%)
  지번                            : 고유값 311,629개 | NULL  8,228건 (  0.1%)
  지목명                           : 고유값     28개 | NULL      0건 (  0.0%)
  토지면적                          : 고유값 115,593개 | NULL      0건 (  0.0%)
  용도지역명1                        : 고유값     24개 | NULL      0건 (  0.0%)
  용도지역명2                        : 고유값     24개 | NULL      0건 (  0.0%)
  토지이용상황                        : 고유값     55개 | NULL      0건 (  0.0%)
  지형높이                          : 고유값      6개 | NULL      0건 (  0.0%)
  지형형상                          : 고유값

### 6-1. query_key 개선 (읽기 쉬운 형태로 변환)

In [39]:
print("=" * 80)
print("🔧 토지특성정보 전처리: 주소_텍스트 생성")
print("=" * 80)

# 기존 query_key 형태 확인
print("\n기존 query_key 샘플:")
print(land_df['query_key'].head(5).tolist())

# 새로운 주소_텍스트 생성: "법정동명 + 지번"
# 예: "서울특별시 종로구 청운동 1" 또는 "지번없음"
def create_readable_query_key(row):
    """
    읽기 쉬운 주소_텍스트 생성
    - 지번 있음: "법정동명 + 지번"
    - 지번 없음: "지번없음" (아직 지번을 받지 못한 지역)
    """
    base = row['법정동명']
    
    if pd.notna(row['지번']):
        jibun = str(row['지번'])
        return f"{base} {jibun}"
    else:
        # 지번이 없는 경우 명시적으로 표시
        return "지번없음"

# 새로운 컬럼 생성
land_df['주소_텍스트'] = land_df.apply(create_readable_query_key, axis=1)

print("\n✅ 주소_텍스트 생성 완료")
print("\n변환 예시 (처음 10건):")
comparison = land_df[['법정동명', '지번', 'query_key', '주소_텍스트']].head(10)
for idx, row in comparison.iterrows():
    print(f"\n{idx+1}.")
    print(f"  법정동명: {row['법정동명']}")
    print(f"  지번: {row['지번'] if pd.notna(row['지번']) else 'NULL'}")
    print(f"  기존 query_key: {row['query_key']}")
    print(f"  신규 주소_텍스트: {row['주소_텍스트']}")

# 생성 통계
print(f"\n📊 생성 통계:")
print(f"  - 전체: {len(land_df):,}건")
print(f"  - 지번 있음: {land_df['지번'].notna().sum():,}건 ({land_df['지번'].notna().sum()/len(land_df)*100:.1f}%)")
print(f"  - 지번 없음: {land_df['지번'].isna().sum():,}건 ({land_df['지번'].isna().sum()/len(land_df)*100:.1f}%)")

# 지번 없는 케이스 샘플
print(f"\n📝 지번 없는 케이스 샘플 (5건):")
no_jibun_samples = land_df[land_df['지번'].isna()][['법정동명', '주소_텍스트']].head(5)
for idx, row in no_jibun_samples.iterrows():
    print(f"  {idx+1}. {row['법정동명']} → {row['주소_텍스트']}")

# 지번 있는 케이스 샘플
print(f"\n📝 지번 있는 케이스 샘플 (5건):")
with_jibun_samples = land_df[land_df['지번'].notna()][['법정동명', '지번', '주소_텍스트']].head(5)
for idx, row in with_jibun_samples.iterrows():
    print(f"  {idx+1}. {row['법정동명']} {row['지번']} → {row['주소_텍스트']}")

# query_key 컬럼 삭제 (주소_텍스트로 대체)
land_df = land_df.drop(columns=['query_key'])
print(f"\n✅ 기존 query_key 컬럼 삭제 완료")


🔧 토지특성정보 전처리: 주소_텍스트 생성

기존 query_key 샘플:
['1111010100|1', '1111010100|1-1', '1111010100|1-2', '1111010100|1-3', '1111010100|1-4']


KeyboardInterrupt: 

### 6-2. 토지특성정보 저장

In [ ]:
print("=" * 80)
print("💾 토지특성정보 저장")
print("=" * 80)

# 저장할 파일명
land_output_file = data_dir / '토지특성정보_전처리완료.csv'

# CSV 저장
land_df.to_csv(land_output_file, index=False, encoding='utf-8-sig')

print(f"\n✅ 저장 완료: {land_output_file}")
print(f"\n파일 정보:")
print(f"  - 레코드 수: {len(land_df):,}건")
print(f"  - 컬럼 수: {len(land_df.columns)}개")
print(f"  - 파일 크기: {land_output_file.stat().st_size / 1024 / 1024:.1f} MB")

print(f"\n📋 최종 컬럼 목록 ({len(land_df.columns)}개):")
for i, col in enumerate(land_df.columns, 1):
    print(f"  {i:2d}. {col}")


💾 토지특성정보 저장

✅ 저장 완료: Backend\target\classes\data\토지특성정보_전처리완료.csv

파일 정보:
  - 레코드 수: 5,841,461건
  - 컬럼 수: 15개
  - 파일 크기: 1258.0 MB

📋 최종 컬럼 목록 (15개):
   1. 법정동코드
   2. 법정동명
   3. 대장구분명
   4. 지번
   5. 지목명
   6. 토지면적
   7. 용도지역명1
   8. 용도지역명2
   9. 토지이용상황
  10. 지형높이
  11. 지형형상
  12. 도로접면
  13. query_key
  14. 구분코드
  15. 주소_텍스트


## 7. 전처리 최종 결과 요약

In [ ]:
print("=" * 80)
print("📊 전처리 최종 결과 요약")
print("=" * 80)

print("\n" + "="*80)
print("1️⃣ 법규조례 데이터 (Law)")
print("="*80)

print(f"\n📈 기본 정보:")
print(f"  - 원본 레코드: 913,591건")
print(f"  - 전처리 후: {len(law_df):,}건")
print(f"  - 최종 컬럼: {len(law_df.columns)}개")

print(f"\n🔧 수행한 전처리:")
print(f"  1. 지역명 추가 (구분코드 → 지역명 매핑)")
print(f"     - 매핑 성공률: 100%")
print(f"     - 예: 11110 → '서울특별시 종로구'")
print(f"  ")
print(f"  2. 가능여부 정규화 (477개 → 3개 카테고리)")
print(f"     - 가능: {law_df[law_df['가능여부_정규화']=='가능'].shape[0]:,}건 (62.5%)")
print(f"     - 불가: {law_df[law_df['가능여부_정규화']=='불가'].shape[0]:,}건 (37.5%)")
print(f"     - 알수없음: {law_df[law_df['가능여부_정규화']=='알수없음'].shape[0]:,}건 (0.0%)")

print(f"\n📋 최종 컬럼 (7개):")
for col in ['구분코드', '지역명', '법률명', '용도지역지구명', '토지이용명', '가능여부_정규화', '조건제한예외사항']:
    print(f"  - {col}")

print(f"\n📝 샘플 데이터:")
print(law_df[['지역명', '용도지역지구명', '토지이용명', '가능여부_정규화']].head(3))

print(f"\n💾 저장 파일:")
print(f"  - Backend/target/classes/data/법규조례_전처리완료.csv")

print("\n" + "="*80)
print("2️⃣ 토지특성정보 데이터 (LandChar)")
print("="*80)

print(f"\n📈 기본 정보:")
print(f"  - 원본 레코드: 5,841,461건")
print(f"  - 전처리 후: {len(land_df):,}건")
print(f"  - 최종 컬럼: {len(land_df.columns)}개")

print(f"\n🔧 수행한 전처리:")
print(f"  1. 주소_텍스트 생성 (query_key 대체)")
print(f"     - 지번 있음: {land_df[land_df['주소_텍스트']!='지번없음'].shape[0]:,}건 (99.9%)")
print(f"     - 지번 없음: {land_df[land_df['주소_텍스트']=='지번없음'].shape[0]:,}건 (0.1%)")
print(f"     - 예: '서울특별시 종로구 청운동 1'")
print(f"  ")
print(f"  2. query_key 컬럼 삭제 (주소_텍스트로 대체)")

print(f"\n📋 주요 컬럼:")
for col in ['법정동코드', '법정동명', '지번', '용도지역명1', '토지이용상황', '주소_텍스트', '구분코드']:
    if col in land_df.columns:
        print(f"  - {col}")

print(f"\n📝 샘플 데이터:")
print(land_df[['법정동명', '지번', '용도지역명1', '주소_텍스트']].head(3))

print(f"\n💾 저장 파일:")
print(f"  - Backend/target/classes/data/토지특성정보_전처리완료.csv")

print("\n" + "="*80)
print("✅ 전체 전처리 완료!")
print("="*80)

print(f"\n📌 다음 단계:")
print(f"  1. Spring Boot/FastAPI에서 전처리 완료 파일 로드")
print(f"  2. 챗봇 RAG 시스템 구축")
print(f"  3. pgvector 임베딩 및 검색 테스트")
print(f"  4. 법/조례 + 토지특성 조인 쿼리 구현")


📊 전처리 최종 결과 요약

1️⃣ 법규조례 데이터 (Law)

📈 기본 정보:
  - 원본 레코드: 913,591건
  - 전처리 후: 913,591건
  - 최종 컬럼: 8개

🔧 수행한 전처리:
  1. 지역명 추가 (구분코드 → 지역명 매핑)
     - 매핑 성공률: 100%
     - 예: 11110 → '서울특별시 종로구'
  
  2. 가능여부 정규화 (477개 → 3개 카테고리)
     - 가능: 570,926건 (62.5%)
     - 불가: 342,605건 (37.5%)
     - 알수없음: 60건 (0.0%)

📋 최종 컬럼 (7개):
  - 구분코드
  - 지역명
  - 법률명
  - 용도지역지구명
  - 토지이용명
  - 가능여부_정규화
  - 조건제한예외사항

📝 샘플 데이터:
  지역명 용도지역지구명         토지이용명 가능여부_정규화
0  전국  산업정비구역  건축물(가설건축물포함)       가능
1  전국  산업정비구역           공작물       가능
2  전국  산업정비구역            토지       가능

💾 저장 파일:
  - Backend/target/classes/data/법규조례_전처리완료.csv

2️⃣ 토지특성정보 데이터 (LandChar)

📈 기본 정보:
  - 원본 레코드: 5,841,461건
  - 전처리 후: 5,841,461건
  - 최종 컬럼: 15개

🔧 수행한 전처리:
  1. 주소_텍스트 생성 (query_key 대체)
     - 지번 있음: 5,833,233건 (99.9%)
     - 지번 없음: 8,228건 (0.1%)
     - 예: '서울특별시 종로구 청운동 1'
  
  2. query_key 컬럼 삭제 (주소_텍스트로 대체)

📋 주요 컬럼:
  - 법정동코드
  - 법정동명
  - 지번
  - 용도지역명1
  - 토지이용상황
  - 주소_텍스트
  - 구분코드

📝 샘플 데이터:
            법정동명   지번     용도지역명1      